In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None) #display all columns
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv("../input/league-of-legends-diamond-ranked-games-10-min/high_diamond_ranked_10min.csv") #read csv file

In [ ]:
df.head(5) 

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df=df.drop(columns="gameId")
df=df.rename(columns={"blueWins": "target"})

Tell you where is the NAN value but in this case, there is no nan value

In [ ]:
for columname in df.columns:
    if df[columname].count() != len(df):
         loc = data[columname][data[columname].isnull().values==True].index.tolist()
         print('index："{}", Column{} missing value'.format(columname,loc))

In [ ]:
df.groupby('target').describe()

In [ ]:
df.corr()

In [ ]:
corrMatrix = df.corr()

In [ ]:
cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

corrMatrix.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '40px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())

In [ ]:
df.groupby('target').describe()

categorica and numerical, so we can use later

In [ ]:
features_categorical=[]
features_numerical=[i for i in df.columns if i not in features_categorical]
features_numerical.remove("target")
print(features_categorical)
print(features_numerical)

In [ ]:
for i in df.columns:
    print("========================================")
    print("How many uniques?"+"  "+str(len(df[i].unique())))
    print(i, ' : ', df[i].unique())

In [ ]:
unique_lessthan10=[]
for i in df.columns:
    if len(df[i].unique())<=10:
        unique_lessthan10.append(i)
print(unique_lessthan10)


In [ ]:
for j in unique_lessthan10:
    sns.catplot(y=j, data=df, hue='target',kind='count')
    plt.figure(figsize=(10,5))
    plt.show()

In [ ]:
df[features_numerical].hist(figsize=(30,20))
None

based on deep understading of the games, so we choose KDA

In [ ]:
df["blueKDA"]=(df.blueKills+df.blueAssists)/df.blueDeaths
df["redKDA"]=(df.redKills+df.redAssists)/df.redDeaths

In [ ]:
df[['blueKDA','redKDA']].describe()

In [ ]:
df=df.replace(np.inf, 0)

In [ ]:
df[['blueKDA','redKDA']].describe()

In [ ]:
df=df.drop(columns=["blueKills","blueAssists","blueDeaths","redKills","redAssists","redDeaths"])

In [ ]:
df.head(5)

In [ ]:
df.groupby(df[['blueFirstBlood', 'redFirstBlood']].apply(lambda x: (x['blueFirstBlood'] + x['redFirstBlood'])==1, axis=1))["target"].count()

In [ ]:
df=df.drop(columns="redFirstBlood")

In [ ]:
df.groupby(df[['blueDragons', 'redDragons']].apply(lambda x: (x['blueDragons'] + x['redDragons'])==1, axis=1))["target"].count()

In [ ]:
df.groupby(df[['blueHeralds', 'redHeralds']].apply(lambda x: (x['blueHeralds'] + x['redHeralds'])==1, axis=1))["target"].count()

In [ ]:
df.groupby(df[['blueTotalGold', 'redTotalGold',"blueGoldDiff"]].apply(lambda x: (x['blueTotalGold'] - x['redTotalGold'])==x['blueGoldDiff'], axis=1))["target"].count()

In [ ]:
df=df.drop(columns=["blueTotalGold","redTotalGold","redGoldDiff"])

In [ ]:
df.groupby(df[['blueAvgLevel', 'blueTotalExperience',"redAvgLevel","redTotalExperience"]].apply(lambda x: (x['blueTotalExperience']/ x['blueAvgLevel'])==(x['redTotalExperience']/x['redAvgLevel']), axis=1))["target"].count()

In [ ]:
df.groupby(df[['blueTotalExperience', 'redTotalExperience',"blueExperienceDiff"]].apply(lambda x: (x['blueTotalExperience'] - x['redTotalExperience'])==x['blueExperienceDiff'], axis=1))["target"].count()

In [ ]:
df=df.drop(columns=["blueTotalExperience","redTotalExperience","redExperienceDiff"])

In [ ]:
df.head(5)

Start model training

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X=df.drop(columns="target")
y=df['target']

In [ ]:
X.shape,y.shape

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.2)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as knnc
from sklearn.tree import DecisionTreeClassifier as dtc
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB as gnb
from sklearn.model_selection import cross_val_score

SVM linear need take too long time.

In [ ]:
classifiers = [
    knnc(n_jobs=-1),
    dtc(),
    SVC(),   
    gnb(),
    #SVC(kernel='linear')
]
classifier_names = [
    'K nearest neighbors',
    'Decision Tree Classifier',
    'SVM classifier with RBF kernel',
    'Gaussian Naive Bayes',
    #'SVM classifier with linear kernel'
]

In [ ]:
for clf, clf_name in zip(classifiers, classifier_names):
    cv_scores = cross_val_score(clf, X_train, y_train, cv=5)
    print(clf_name, ' mean accuracy: ', round(cv_scores.mean()*100, 3), '% std: ', round(cv_scores.var()*100, 3),'%')

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Logistic Regression accuracy score with all the features: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)

In [ ]:
y_pre = regressor.predict(X_test)    
accuracy = np.mean(y_pre.round() == y_test.ravel()) * 100
print ("accuracy:",accuracy)

In [ ]:
df_tupe = pd.DataFrame(zip(df.columns,regressor.coef_))
df_tupe.sort_values(by=[1],ascending=False)[:10]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier()
rfc = rfc.fit(X_train, y_train.ravel())
y_predicted = rfc.predict(X_test)    
accuracy = np.mean(y_predicted == y_test) * 100


print ("accuracy:",accuracy)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train.ravel())

In [ ]:
rf_random.best_params_

In [ ]:
best_random = rf_random.best_estimator_

In [ ]:
y_predicted = best_random.predict(X_test)    
accuracy = np.mean(y_predicted == y_test.ravel()) * 100

print ("accuracy:",accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:

y_predict_RandomForestClassifier = best_random.predict(X_test)

print('test accuracy for Random Forest classifier:'\
      , round(accuracy_score(y_test, y_predict_RandomForestClassifier)*100, 2), '%')
plt.title('Confusion matrix for RandomForest classifier')
sns.heatmap(confusion_matrix(y_test.ravel(), y_predict_RandomForestClassifier), annot=True, cmap="YlGn")
plt.xlabel('Predicted classes')
plt.ylabel('True Classes')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
y_scores_RandomForestClassifier=best_random.predict_proba(X_test)
fpr_rfc, tpr_rfc, thresholds_rfc = roc_curve(y_test.ravel(), y_scores_RandomForestClassifier[:,1])
plt.plot(fpr_rfc, tpr_rfc, label="ROC Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positve Rate (recall)")
# find threshold closest to zero
close_zero = np.argmin(np.abs(thresholds_rfc))
plt.plot(fpr_rfc[close_zero], tpr_rfc[close_zero], 'o', markersize=10,
label="threshold_rfc", fillstyle="none", c='k', mew=2)
plt.legend(loc=4)